In [1]:
# testing objects for n-dim tead

In [2]:
import torch
from botorch.models.gp_regression import ExactGP
from scipy.stats.qmc import LatinHypercube
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [3]:
def nfinite_diff(model):
    """get approx gradient at model training points"""
    # assumes 1d input... see implementation in [3] for update to n-D
    h = 1e-4
    x = model.train_inputs[0].squeeze()
    m = len(x)
    delta_s = torch.zeros(m)
    for i in range(0, m):
        x_lo = x[i] - h
        if x_lo < 0.0:
            x_lo = torch.DoubleTensor([0.0])
        x_hi = x[i] + h
        if x_hi > 1.0:
            x_hi = torch.DoubleTensor([1.0])
        y_lo = model(x_lo.unsqueeze(-1)).mean
        y_hi = model(x_hi.unsqueeze(-1)).mean
        delta_s[i] = (y_hi - y_lo)/(2*h)
    return delta_s

In [5]:
dtype = torch.double
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_mll

train_X = torch.rand(20, 2, dtype=dtype)
train_Y = torch.sin(train_X).sum(dim=1, keepdim=True)
model_example = SingleTaskGP(train_X, train_Y)
mll_example = ExactMarginalLogLikelihood(model_example.likelihood, model_example)
fitting = fit_gpytorch_mll(mll_example)

In [6]:
temp = nfinite_diff(model_example)

RuntimeError: Boolean value of Tensor with more than one value is ambiguous